In [5]:
# Step 1: Install if needed (run once)
# !pip install yfinance pandas numpy matplotlib seaborn

# Step 2: Import libraries
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style='whitegrid')


In [2]:
# Step 3: Define tickers - NIFTY 50 example (or US tickers if preferred)
tickers = ['TCS.NS', 'INFY.NS', 'RELIANCE.NS', 'HDFCBANK.NS', 'ITC.NS']

# Step 4: Download historical data
data = yf.download(tickers, start="2020-01-01", end="2024-12-31", group_by='ticker')


C:\Users\soumy\AppData\Local\Temp\ipykernel_28088\914789724.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start="2020-01-01", end="2024-12-31", group_by='ticker')
[*********************100%***********************]  5 of 5 completed


In [3]:
!pip install seaborn


Defaulting to user installation because normal site-packages is not writeable


In [4]:
import seaborn as sns
sns.set(style='whitegrid')
